<a href="https://colab.research.google.com/github/SEONGHYUN99/ML-for-AI/blob/main/221006.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
cd /content/MyDrive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04

/content/MyDrive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04


In [ ]:
import pandas as pd
import numpy as np

## Task1
- Practice ch4-2 codes
- Find which quarter had the most rain from the daily 
precipitation data
- Assign ‘quarterly total precipitation’ and ‘number of 
days in a month’ variables to daily precipitation data.
- Calculate rolling 3day mean precipitation


In [ ]:
weather = pd.read_csv('data/weather_by_station.csv', index_col='date', parse_dates=True)
weather.head()

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1CTFR0039,0.0,"STAMFORD 4.2 S, CT US"
2018-01-01,PRCP,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0015,0.0,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,PRCP,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.0,"GLEN ROCK 0.7 SSE, NJ US"


In [ ]:
dp=weather.query('datatype=="PRCP"').groupby(level=0).mean()
dp

,value
date,
2018-01-01,0.000000
2018-01-02,0.000000
2018-01-03,0.000000
2018-01-04,4.916071
2018-01-05,12.303571
...,...
2018-12-27,0.000000
2018-12-28,10.462025
2018-12-29,18.655263


In [ ]:
weather.query('datatype == "PRCP"')\
    .groupby(level=0).mean()\
    .groupby(pd.Grouper(freq='Q')).sum().value.nlargest()

date
2018-09-30    546.518380
2018-12-31    473.972345
2018-03-31    364.885769
2018-06-30    334.893094
Name: value, dtype: float64

In [ ]:
dp.assign(
        total_prcp_in_Q=lambda x: \
            x.groupby(pd.Grouper(freq='Q')).transform(np.sum),
        total_day_count_in_Q=lambda x:\
        x.groupby(pd.Grouper(freq='Q')).value.transform('count'),
        pct_monthly_prcp=lambda x:\
        x.value.div(x.total_prcp_in_Q)
    )\

,value,total_prcp_in_Q,total_day_count_in_Q,pct_monthly_prcp
date,,,,
2018-01-01,0.000000,364.885769,90,0.000000
2018-01-02,0.000000,364.885769,90,0.000000
2018-01-03,0.000000,364.885769,90,0.000000
2018-01-04,4.916071,364.885769,90,0.013473
2018-01-05,12.303571,364.885769,90,0.033719
...,...,...,...,...
2018-12-27,0.000000,473.972345,92,0.000000
2018-12-28,10.462025,473.972345,92,0.022073
2018-12-29,18.655263,473.972345,92,0.039359


## Task2
- Practice crosstab and pivot_table
- Use pivot_table to produce the same result as below


In [ ]:
fb = pd.read_csv('data/fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


In [ ]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'] # name the columns index
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


In [ ]:
fb.pivot_table(
     index="trading_volume",
    columns=fb.index.month,
   values='close',aggfunc="count"
    
)

date,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0
